In [1]:
using SwiftObjectStores

In [2]:
using ProgressMeter

In [14]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams
using DataStructures
using Distances
using Iterators
using StatsBase
using Training
using AdaGramCompat

In [4]:
SwiftObjectStores.list(SwiftService(), "sensemodels")

Dict{Pair{Any,Any},Pair{Any,Any}} with 1 entry:
  Pair{Any,Any}("action","list_container_part") => Pair{Any,Any}("marker","")

In [18]:
function all_word_sense_vectors(ee::WordEmbedding, word)
    if haskey(ee.embedding, word)
        [ee.embedding[word]]
    else
        Vector{Float32}[]
    end
end


function all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

function all_word_sense_vectors(am::AdaGramCompat.AdaGramModel, word)
    if haskey(am.dict.word2id, word)
        wsv_mat = word_sense_vectors(am, word)
        [view(wsv_mat,:,ii) for ii in 1:size(wsv_mat,2)]
    else
        Vector{Float32}[]
    end
end

all_word_sense_vectors (generic function with 3 methods)

In [5]:
using JLD
#ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.jld","ee");
eep = get_jld(SwiftService(), "sensemodels/plain/", "tokenised_lowercase_WestburyLab.wikicorp.201004_100_nosubsample.jld", "ee");


In [ ]:
using AdaGram
using AdaGramCompat
am = load("models/adagram/more_senses.adagram_model.jld", "am");

In [50]:
s_am = get_jld(SwiftService(), "sensemodels/adagram/", "semhuff_more_senses.adagram_model.jld", "am");

In [54]:
function load_wordsim353(filename="data/corpora/wordsim353/combined.csv")
    wordsims = readdlm(filename, ','; skipstart=1)
    wordpairs = convert(Matrix{String},wordsims[:,1:2])
    groundsim = convert(Vector{Float64},wordsims[:,3])
    (wordpairs, groundsim)
end


load_wordsim353 (generic function with 2 methods)

In [55]:
function get_sims(ee, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1::Matrix{Float32} = hcat(all_word_sense_vectors(ee,wordpairs[ii,1])...)
            wvs_2::Matrix{Float32} = hcat(all_word_sense_vectors(ee,wordpairs[ii,2])...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end


get_sims (generic function with 3 methods)

In [56]:
wordpairs, groundsim = load_wordsim353();

sims = get_sims(ee,map(lowercase,wordpairs))
corspearman(groundsim,sims)

0.5454215969351677

In [57]:
sims = get_sims(am,map(lowercase,wordpairs))
corspearman(groundsim,sims)

0.46129108146748254

In [58]:
function load_SCWS(filename = "data/corpora/SCWS/ratings.txt")
    function get_context2(fulltext)
        Task() do
            skip_next = 0
            for word in split(fulltext, ' ')
                skip_next-=1
                if word == "<b>"
                    skip_next=3 
                end
                skip_next>0 && continue
                produce(lowercase(word))
            end
        end |> collect
    end
    entries = readdlm(filename, '\t', String; quotes=false, comments=false)
    wordpairs = map(lowercase,entries[:,[2,4]])
    groundsim = map(s->parse(Float64,s), entries[:,8])
    contexts = map(get_context2,entries[:,[6,7]])
    (wordpairs,contexts,groundsim,)
end


load_SCWS (generic function with 2 methods)

In [59]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            
            sense1 = WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end


get_sims (generic function with 3 methods)

In [60]:
scws_wordpairs, scws_contexts, scws_groundsim = load_SCWS();
local_sims = get_sims(ee, scws_wordpairs, scws_contexts)
corspearman(scws_groundsim, scws_local_sims)


LoadError: LoadError: MethodError: no method matching get_sims(::WordEmbeddings.WordEmbedding, ::Array{String,2}, ::Array{Array{String,1},2})
Closest candidates are:
  get_sims(!Matched::WordEmbeddings.FixedWordSenseEmbedding, ::Array{String,2}, ::Array{T,2}) at In[59]:2
  get_sims(::Any, ::Array{String,2}) at In[55]:2
  get_sims(!Matched::WordEmbeddings.FixedWordSenseEmbedding, ::Array{String,2}) at In[8]:2
while loading In[60], in expression starting on line 2

In [61]:
global_syms = get_sims(am, scws_wordpairs)
corspearman(scws_groundsim, global_syms)


2016-09-10T22:26:12.034 - warn: MethodError: Cannot `convert` an object of type Array{Any,1} to an object of type Array{Float32,2}
This may have arisen from a call to the constructor Array{Float32,2}(...),
since type constructors fall back to convert methods. prefix: "error: "
2016-09-10T22:26:12.208 - warn: MethodError: Cannot `convert` an object of type Array{Any,1} to an object of type Array{Float32,2}
This may have arisen from a call to the constructor Array{Float32,2}(...),
since type constructors fall back to convert methods. prefix: "error: "


0.45416228327202474

In [79]:

function synthesize_embedding(am::AdaGramModel,context::AbstractVector, word::AbstractString)
    known_context = filter(c->haskey(am.dict.word2id, c), context)
    sum(all_word_sense_vectors(am, word).*disambiguate(am.vm, am.dict, word, known_context))
end

synthesize_embedding (generic function with 2 methods)

In [84]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end


function synthesize_embedding(ee,context::AbstractVector, word_or_phrase::AbstractString)
    words = split(word_or_phrase, " ")
    wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
            throw(KeyError("None of $words have embeddings"))
    end
    logprobs = [Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=true) for wv in wvs]
    weighted_average(logprobs, wvs)
end
    
    

"""
synthesize_embedding:: Embeddings, Context, Word -> WordVector
"""
    function get_sims_synth(ee, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    npairs = size(wordpairs,1)
        @showprogress for ii in 1:npairs
        try           
            wv1 = synthesize_embedding(ee, contexts[ii,1],wordpairs[ii,1])
            wv2 = synthesize_embedding(ee, contexts[ii,2],wordpairs[ii,2])
             
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end


get_sims_synth

In [85]:
corspearman(scws_groundsim, get_sims_synth(ee, scws_wordpairs,scws_contexts))

LoadError: LoadError: AssertionError: 
while loading In[85], in expression starting on line 1

In [86]:
sims = get_sims_synth(am, scws_wordpairs, scws_contexts)
corspearman(scws_groundsim, sims)

Progress:   8%|███                                      |  ETA: 0:00:332016-09-10T23:43:37.49 - warn: KeyError: key "insufflate" not found prefix: "error: "
Progress:  27%|███████████                              |  ETA: 0:00:262016-09-10T23:43:44.487 - warn: KeyError: key "backdate" not found prefix: "error: "
Progress: 100%|█████████████████████████████████████████| Time: 0:00:36


0.2194597743906767

In [83]:
corspearman(scws_groundsim, 1-sims)

-0.22246958780743398

In [ ]:
# 0.5342692139056353     GlobalMin   plain/tokenised_lowercase_WestburyLab.wikicorp.201004_100_nosubsample.jld
# 0.48861817784625977    GlobalMin   adagram/more_senses.adagram_model.jld
# 0.45416228327202474    GlobalMin   sensemodels/adagram/semhuff_more_senses.adagram_model.jld"
# 0.523055629491102      GlobalMin   sensemodels/adagram/semhuff_more_senses.adagram_model.jld"
# 0.524743130161508      LocalSynth   greedy/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.jl
# 0.6452187057383217     LocalSynth   adagram/more_senses.adagram_model.jld